In [9]:
import os
import codecs
import datetime
import zipfile
import pandas as pd
from join_logs import readWords, joinWordsAndVAD, openAndReadLogFile
import sys
import numpy as np 

    
def loadFiles(path="logs_cleaned"):
    furhat_files = []
    
    for filename in os.listdir(path):
        if filename.endswith('Furhat.log'):
            src = path+ "/"+filename
            furhat_files.append(src)
            
    return furhat_files


files = loadFiles()
print(files)

for file in files:
    result = openAndReadLogFile(file,['Log_Type.FURHAT_MESSAGE'])
    df = pd.DataFrame(result)
    df.columns =['date', 'timestamp', 'log_type', 'furhat_action','value','ignore','ignore1'] 
    start_stamp = pd.to_numeric(df[df['value']=='{text: Jag heter föör hatt.}']['timestamp'].values)
    df['timestamp'] = pd.to_numeric(df['timestamp'])
    new_col = df['timestamp']-float(start_stamp)
    df['timestamp'] = new_col
    df = df.drop(['date','log_type','ignore','ignore1'],axis=1)
    #gaze_df = df
    gaze_df = df[df['furhat_action']=='furhatos.event.actions.ActionGaze']
    speech_df = df[df['furhat_action']=='furhatos.event.actions.ActionSpeech']
    gaze_df = gaze_df.drop(['furhat_action'],axis=1).reset_index()
    speech_df = speech_df.drop(['furhat_action'],axis=1).reset_index()

    pos = gaze_df['value'].values
    x = []
    y = []
    side = []
    file_names = []
    for i in range(len(pos)):
        split_var = pos[i].split(',')
        if float(split_var[0][4:]) < 0.0:
            side.append('left')
        elif float(split_var[0][4:]) == 0.0:
            side.append('straight')
        else:
            side.append('right')  
        x.append(split_var[0][4:])
        y.append(split_var[1][4:])
        file_names.append(file)
        
    #gaze_df['file'] = file_names
    gaze_df['x_pos'] = x
    gaze_df['y_pos'] = y
    gaze_df['side'] = side
    gaze_df = gaze_df.drop(['value'],axis=1)
    gaze_df = gaze_df.drop(['index'],axis=1)
    speech_df = speech_df.drop(['index'],axis=1)
    print(speech_df)
    gaze_df.to_csv('furhat_csvs/'+file[13:-4]+'.csv')
    speech_df.to_csv('furhat_speech_csvs/'+file[13:-4]+'_speech.csv')
    

['logs_cleaned/Oct-01-2020_15-04-33_control_096_Furhat.log', 'logs_cleaned/Oct-01-2020_15-59-36_mediation_098_Furhat.log', 'logs_cleaned/Oct-01-2020_17-28-47_control_100_Furhat.log', 'logs_cleaned/Oct-02-2020_13-04-29_control_102_Furhat.log', 'logs_cleaned/Oct-02-2020_13-54-16_mediation_104_Furhat.log', 'logs_cleaned/Oct-02-2020_16-12-07_control_106_Furhat.log', 'logs_cleaned/Oct-02-2020_16-47-18_mediation_108_Furhat.log', 'logs_cleaned/Sep-18-2020_16-01-46_mediation_038_Furhat.log', 'logs_cleaned/Sep-21-2020_15-13-05_control_040_Furhat.log', 'logs_cleaned/Sep-21-2020_16-49-13_mediation_044_Furhat.log', 'logs_cleaned/Sep-23-2020_12-53-50_control_046_Furhat.log', 'logs_cleaned/Sep-23-2020_13-52-17_mediation_048_Furhat.log', 'logs_cleaned/Sep-23-2020_14-37-52_control_050_Furhat.log', 'logs_cleaned/Sep-23-2020_15-57-49_control_054_Furhat.log', 'logs_cleaned/Sep-23-2020_17-26-08_mediation_056_Furhat.log', 'logs_cleaned/Sep-24-2020_15-59-45_mediation_060_Furhat.log', 'logs_cleaned/Sep-24-20